<a href="https://colab.research.google.com/github/Danie261/Trading-Algoritmico/blob/main/Copia_de_Untitled4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# INDICADORES TÉCNICOS

In [ ]:
!pip install yfinance
!pip install talib-binary
!pip install seaborn

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import numpy as np
import talib as ta
import seaborn as sns
sns.set()

ticker="TSLA"
tf="2020-6-1"
info=yf.Ticker(ticker).info
longname=info["longName"]
if "," in longname:
    longname=longname.replace(",","")

##Coding 'AROON' Indicator

In [ ]:
df=yf.download(ticker, tf)

df["Index"]=range(len(df))
n=10


#Cálculo 'AROON'
class aroon():
    def a_up(self):
        a=df["High"].rolling(n).max()
        dif=[]
        aup=[]
        for i in range(len(df)):
            index=0
            if i<n:
                dif+=[np.nan]
                aup+=[np.nan]
            else:
                for j in range(i,i-n,-1):
                    if df["High"][j]==a[i]:
                        index=df["Index"][j]
                dif+=[i-index]
                aup+=[int(((n-(i-index))/n)*100)]
        return aup

    def a_down(self):
        a=df["Low"].rolling(n).min()
        dif=[]
        adown=[]
        for i in range(len(df)):
            index=0
            if i<n:
                dif+=[np.nan]
                adown+=[np.nan]
            else:
                for j in range(i,i-n,-1):
                    if df["Low"][j]==a[i]:
                        index=df["Index"][j]
                dif+=[i-index]
                adown+=[int(((n-(i-index))/n)*100)]
        return adown, dif

x=aroon()       
df["AROON Up"]=x.a_up()        
df["AROON Down"], df["dif"]=x.a_down()


#Algoritmo de c/v
def signals():
    b=[]
    s=[]
    for i in range(len(df)):
        if i==0:
            b+=[np.nan]
            s+=[np.nan]
        elif df["AROON Up"][i-1]<df["AROON Down"][i-1] and df["AROON Up"][i]>df["AROON Down"][i]:
            b+=[df["Close"][i]]
            s+=[np.nan]
        elif df["AROON Up"][i-1]>df["AROON Down"][i-1] and df["AROON Up"][i]<df["AROON Down"][i]:
            b+=[np.nan]
            s+=[df["Close"][i]]
        else:
            b+=[np.nan]
            s+=[np.nan]
    return b, s

df["Buy"], df["Sell"]=signals()


#Plotting
plt.figure(figsize=(17,11))
plt.subplot(211)
plt.plot(df["Close"], label="Stock Price", alpha=0.7)
plt.scatter(df.index, df["Buy"], label="Buy Price", marker="^", color="green")
plt.scatter(df.index, df["Sell"], label="Sell Price", marker="v", color="red")
plt.title(f"{longname} Stock Price")
plt.legend(loc="upper left")


plt.subplot(212)
plt.plot(df["AROON Up"], label="AROON Up")
plt.plot(df["AROON Down"], label="AROON Down")
plt.axhline(50, color="black", lw=0.5)
plt.axhline(70, color="black", lw=0.5)
plt.axhline(30, color="black", lw=0.5)
plt.title("AROON Chart")
plt.xlabel("Date")
plt.legend(loc="upper left")

plt.show()

Comparación con TA-Lib

In [ ]:
da=pd.DataFrame()

da["AROON Down"]=df["AROON Down"]
da["AROON Up"]=df["AROON Up"]

da["AROON Down (TA-Lib)"], da["AROON Up (TA-Lib)"]=ta.AROON(df["High"], df["Low"], n)
da.dropna(inplace=True)


#Plotting
plt.figure(figsize=(17,11))
plt.subplot(211)
plt.plot(da["AROON Up"], label="AROON Up")
plt.plot(da["AROON Down"], label="AROON Down")
plt.axhline(50, color="black", lw=0.5)
plt.title("AROON Chart")
plt.legend(loc="upper left")

plt.subplot(212)
plt.plot(da["AROON Up (TA-Lib)"], label="AROON Up (TA-Lib)")
plt.plot(da["AROON Down (TA-Lib)"], label="AROON Down (TA-Lib)")
plt.axhline(50, color="black", lw=0.5)
plt.title("AROON (TA-Lib) Chart")
plt.legend(loc="upper left")
plt.xlabel("Date")

plt.show()
print(da.head(20))

In [ ]:
df=yf.download(ticker, tf)

#Cálculo 'ATR'
class calculo_ATR:
    t=14

    def tr(self):
        tr=[]
        for i in range(len(df)):
            if i!=0:
                a=df["High"][i]-df["Low"][i]
                b=abs(df["High"][i]-df["Close"][i-1])
                c=abs(df["Low"][i]-df["Close"][i-1])
                tr+=[max(a,b,c)]
            else:
                tr+=[np.nan]
        return tr

    def a_tr(self):
        atr=[]
        atr_0=0
        for i in range(len(df)):
            if len(atr)==0:
                for j in range(1,self.t+1):
                    atr_0+=t_r[j]
                atr+=[atr_0/self.t]
            elif len(atr)==1:
                atr+=[(atr[0]*(self.t-1)+t_r[i])/self.t]
            else:
                atr+=[(atr[i-1]*(self.t-1)+t_r[i])/self.t]
        return atr

x=calculo_ATR()
t_r=x.tr()
df["ATR"]=x.a_tr()


#Cálculo '+DM y -DM'
def dms():
    pos_dm=[]
    neg_dm=[]
    for i in range(len(df)):
        if i!=0:
            upmove=df["High"][i]-df["High"][i-1]
            downmove=df["Low"][i]-df["Low"][i-1]
            if upmove>downmove:
                if upmove>0:
                    pos_dm+=[upmove]
                    neg_dm+=[0]
                else:
                    pos_dm+=[0]
                    neg_dm+=[0]
            elif upmove<downmove:
                if downmove>0:
                    neg_dm+=[downmove]
                    pos_dm+=[0]
                else:
                    neg_dm+=[0]
                    pos_dm+=[0]
            else:
                pos_dm+=[0]
                neg_dm+=[0]
        else:
            pos_dm+=[np.nan]
            neg_dm+=[np.nan]
    return pos_dm, neg_dm

df["+DM"], df["-DM"]=dms()

del(df["Open"],df["Adj Close"], df["Volume"])


#Cálculo '+DI y -DI'
t=14
def dis():
    pos_di=[]
    neg_di=[]
    a=df["+DM"].ewm(span=t,adjust=False, min_periods=t).mean()
    b=df["-DM"].ewm(span=t,adjust=False, min_periods=t).mean()

    for i in range(len(df)):
        if i>t-1:
            pos_di+=[(a[i]/df["ATR"][i])*100]
            neg_di+=[abs((b[i]/df["ATR"][i])*100)]
        else:
            pos_di+=[np.nan]
            neg_di+=[np.nan]

    return pos_di, neg_di

df["+DI"], df["-DI"]=dis()


#Cálculo 'DX'
def dx():
    dx=[]
    for i in range(len(df)):
        if i>t-1:
            dx+=[abs((df["+DI"][i]-df["-DI"][i])/(df["+DI"][i]+df["-DI"][i]))*100]
        else:
            dx+=[np.nan]
    return dx

df["DX"]=dx()
df.dropna(inplace=True)


#Cálculo de 'ADX'
def adx():
    adx=[]
    for i in range(len(df)):
        if len(adx)==0:
            adx_0=0
            for j in range(1, t+1):
                adx_0+=df["DX"][j]
            adx+=[adx_0/t]
        
        elif len(adx)==1:
            adx+=[(adx[0]*(t-1)+df["DX"][i])/t]
        else:
            adx+=[(adx[i-1]*(t-1)+df["DX"][i])/t]
    return adx

df["ADX"]=adx()


#Algoritmo de c/v
def signals():
    b=[]
    s=[]
    condition=0
    for i in range(len(df)):
        if df["ADX"][i-1]<25 and df["ADX"][i]>25 and df["+DI"][i]>df["-DI"][i]:
            if condition!=1:
                b+=[df["Close"][i]]
                s+=[np.nan]
                condition=1
                
            else:
                b+=[np.nan]
                s+=[np.nan]
                
        elif df["ADX"][i-1]<25 and df["ADX"][i]>25 and df["-DI"][i]>df["+DI"][i]:
            if condition!=-1:
                b+=[np.nan]
                s+=[df["Close"][i]]
                condition=-1
                
            else:
                b+=[np.nan]
                s+=[np.nan]
                
        else:
            b+=[np.nan]
            s+=[np.nan]
            
    return b, s

b,s=signals()


#Plotting
plt.figure(figsize=(17,11))
plt.subplot(211)
plt.plot(df["Close"], label="Stock Price", alpha=0.8)
plt.scatter(df.index, b, label="Buy Price", marker="^", color="green")
plt.scatter(df.index, s, label="Sell Price", marker="v", color="red")
plt.title(f"{longname} Stock Price")
plt.legend(loc="upper left")

plt.subplot(212)
plt.plot(df["ADX"], color="darkblue", label="ADX", alpha=0.5)
plt.plot(df["+DI"], color="green", label="+DI")
plt.plot(df["-DI"], color="red", label="-DI")
plt.axhline(25, color="black", linestyle ="--")
plt.xlabel("Date")
plt.legend(loc="upper left")
plt.title("ADX Chart")

plt.show()

##Coding 'ATR', 'Keltner Channel' and 'ADX' Indicators

Coding 'ATR'

In [ ]:
data=yf.download(ticker, tf)

df=pd.DataFrame()
df["Close"]=data["Close"]


#Cálculo de 'ATR'
class calculo_ATR:
    t=10

    def tr(self):
        tr=[]
        for i in range(len(data)):
            if i!=0:
                a=data["High"][i]-data["Low"][i]
                b=abs(data["High"][i]-df["Close"][i-1])
                c=abs(data["Low"][i]-df["Close"][i-1])
                tr+=[max(a,b,c)]
            else:
                tr+=[np.nan]
        return tr

    def a_tr(self):
        atr=[]
        atr_0=0
        for i in range(len(data)):
            if len(atr)==0:
                for j in range(1,self.t+1):
                    atr_0+=t_r[j]
                atr+=[atr_0/self.t]
            elif len(atr)==1:
                atr+=[(atr[0]*(self.t-1)+t_r[i])/self.t]
            else:
                atr+=[(atr[i-1]*(self.t-1)+t_r[i])/self.t]
        return atr

x=calculo_ATR()
t_r=x.tr()
df["ATR"]=x.a_tr()


#Plotting
plt.figure(figsize=(17,11))
plt.subplot(211)
plt.title(f"{longname} Stock Price")
plt.plot(df["Close"], label="Stock Price")

plt.subplot(212)
plt.title("Average True Range")
plt.plot(df["ATR"], label="Average True Range", color="brown")
plt.xlabel("Date")
plt.legend(loc="upper left")

plt.show()

Comparación con TA-Lib

In [ ]:
da=pd.DataFrame()

da["ATR"]=df["ATR"]
da["ATR (TA-Lib)"]=ta.ATR(data["High"], data["Low"], df["Close"], 10)

da.dropna(inplace=True)


#Plotting
plt.figure(figsize=(17,11))
plt.subplot(211)
plt.title("Average True Range")
plt.plot(da["ATR"], label="Average True Range")
plt.legend(loc="upper left")

plt.subplot(212)
plt.title("Average True Range (TA-lib)")
plt.plot(da["ATR (TA-Lib)"], label="Average True Range (TA-Lib)", color="brown")
plt.xlabel("Date")
plt.legend(loc="upper left")

plt.show()
print(da.head(50))

Coding 'Keltner Channel'

In [ ]:
#Cálculo 'Middle Band'
t=20
df["Middle"]=df["Close"].ewm(span=t,adjust=False, min_periods=t).mean()


#Cálculo 'Upper Band' y 'Lower Band'
df["Upper"]=df["Middle"]+2*df["ATR"]
df["Lower"]=df["Middle"]-2*df["ATR"]


#Algoritmo de c/v
def signals():
    b=[]
    s=[]
    condition=0
    for i in range(len(df)):
        if i<(len(df)-3):
            if df["Close"][i]>df["Upper"][i] and condition!=1:
                if df["Close"][i+2]>df["Upper"][i+2]:
                    b+=[np.nan]
                    s+=[np.nan]
                    condition=0
                else:
                    b+=[np.nan]
                    s+=[df["Close"][i]]
                    condition=1
            elif df["Close"][i]<df["Lower"][i] and condition!=-1:
                b+=[df["Close"][i]]
                s+=[np.nan]
                condition=-1
            else:
              b+=[np.nan]
              s+=[np.nan]
              condition=0
        else:
          b+=[np.nan]
          s+=[np.nan]
          condition=0

    return b,s

df["Buy"], df["Sell"]=signals()


#Plotting
plt.figure(figsize=(17,11))
plt.title(f"Keltner Channel for {longname}")
plt.plot(df["Middle"], label=f"SMA {t}", linestyle="--")
plt.plot(df["Close"], label="Stock Price", alpha=0.4, color="black")
plt.plot(df["Upper"], label="Upper Band", color="darkblue")
plt.plot(df["Lower"], label="Lower Band", color="red")
plt.fill_between(df.index, df["Lower"], df["Upper"], where=df["Upper"]>df["Lower"], color="lightgreen")
plt.scatter(df.index, df["Buy"], label="Buy Price", marker="^", color="green")
plt.scatter(df.index, df["Sell"], label="Sell Price", marker="v", color="red")
plt.xlabel("Date")
plt.legend(loc="upper left")

plt.show()

Coding 'ADX'

In [ ]:
#Cálculo '+DM y -DM'
def dms():
    pos_dm=[]
    neg_dm=[]
    for i in range(len(df)):
        if i!=0:
            upmove=data["High"][i]-data["High"][i-1]
            downmove=data["Low"][i]-data["Low"][i-1]
            if upmove>downmove:
                if upmove>0:
                    pos_dm+=[upmove]
                    neg_dm+=[0]
                else:
                    pos_dm+=[0]
                    neg_dm+=[0]
            elif upmove<downmove:
                if downmove>0:
                    neg_dm+=[downmove]
                    pos_dm+=[0]
                else:
                    neg_dm+=[0]
                    pos_dm+=[0]
            else:
                pos_dm+=[0]
                neg_dm+=[0]
        else:
            pos_dm+=[np.nan]
            neg_dm+=[np.nan]
    return pos_dm, neg_dm

df["+DM"], df["-DM"]=dms()

del(data["Open"],data["Adj Close"], data["Volume"])


#Cálculo '+DI y -DI'
t=14
def dis():
    pos_di=[]
    neg_di=[]
    a=df["+DM"].ewm(span=t,adjust=False, min_periods=t).mean()
    b=df["-DM"].ewm(span=t,adjust=False, min_periods=t).mean()

    for i in range(len(df)):
        if i>t-1:
            pos_di+=[(a[i]/df["ATR"][i])*100]
            neg_di+=[abs((b[i]/df["ATR"][i])*100)]
        else:
            pos_di+=[np.nan]
            neg_di+=[np.nan]

    return pos_di, neg_di

df["+DI"], df["-DI"]=dis()


#Cálculo 'DX'
def dx():
    dx=[]
    for i in range(len(df)):
        if i>t-1:
            dx+=[abs((df["+DI"][i]-df["-DI"][i])/(df["+DI"][i]+df["-DI"][i]))*100]
        else:
            dx+=[np.nan]
    return dx

df["DX"]=dx()
df.dropna(inplace=True)


#Cálculo de 'ADX'
def adx():
    adx=[]
    for i in range(len(df)):
        if len(adx)==0:
            adx_0=0
            for j in range(1, t+1):
                adx_0+=df["DX"][j]
            adx+=[adx_0/t]
        
        elif len(adx)==1:
            adx+=[(adx[0]*(t-1)+df["DX"][i])/t]
        else:
            adx+=[(adx[i-1]*(t-1)+df["DX"][i])/t]
    return adx

df["ADX"]=adx()


#Algoritmo de c/v
def signals():
    b=[]
    s=[]
    condition=0
    for i in range(len(df)):
        if df["ADX"][i-1]<25 and df["ADX"][i]>25 and df["+DI"][i]>df["-DI"][i]:
            if condition!=1:
                b+=[df["Close"][i]]
                s+=[np.nan]
                condition=1
                
            else:
                b+=[np.nan]
                s+=[np.nan]
                
        elif df["ADX"][i-1]<25 and df["ADX"][i]>25 and df["-DI"][i]>df["+DI"][i]:
            if condition!=-1:
                b+=[np.nan]
                s+=[df["Close"][i]]
                condition=-1
                
            else:
                b+=[np.nan]
                s+=[np.nan]
                
        else:
            b+=[np.nan]
            s+=[np.nan]
            
    return b, s

b,s=signals()


#Plotting
plt.figure(figsize=(17,11))
plt.subplot(211)
plt.plot(df["Close"], label="Stock Price", alpha=0.8)
plt.scatter(df.index, b, label="Buy Price", marker="^", color="green")
plt.scatter(df.index, s, label="Sell Price", marker="v", color="red")
plt.title(f"{longname} Stock Price")
plt.legend(loc="upper left")

plt.subplot(212)
plt.plot(df["ADX"], color="darkblue", label="ADX", alpha=0.5)
plt.plot(df["+DI"], color="green", label="+DI")
plt.plot(df["-DI"], color="red", label="-DI")
plt.axhline(25, color="black", linestyle ="--")
plt.xlabel("Date")
plt.legend(loc="upper left")
plt.title("ADX Chart")

plt.show()

##Coding 'Bollinger Bands' Indicator

In [ ]:
data=yf.download(ticker, tf)

df=pd.DataFrame()
df["Close"]=data["Close"]


#Cálculo 'Middle Band'
t=20
df["SMA"]=df["Close"].rolling(t).mean()


#Cálculo 'Upper Band'
stdev=df["Close"].rolling(t).std()
df["Upper"]=df["SMA"]+2*stdev


#Cálculo 'Lower Band'
df["Lower"]=df["SMA"]-2*stdev


#Algoritmo de c/v
def signals():
    b=[]
    s=[]
    condition=0
    for i in range(len(df)):
        if i<(len(df)-3):
            if df["Close"][i]>df["Upper"][i] and condition!=1:
                if df["Close"][i+2]>df["Upper"][i+2]:
                    b+=[np.nan]
                    s+=[np.nan]
                    condition=0
                else:
                    b+=[np.nan]
                    s+=[df["Close"][i+1]]
                    condition=1
            elif df["Close"][i]<df["Lower"][i] and condition!=-1:
                if df["Close"][i+2]<df["Lower"][i+2]:
                    b+=[np.nan]
                    s+=[np.nan]
                    condition=0
                else:
                    b+=[df["Close"][i+1]]
                    s+=[np.nan]
                    condition=-1
            else:
                b+=[np.nan]
                s+=[np.nan]
                condition=0
        else:
            b+=[np.nan]
            s+=[np.nan]
    return b,s
    
df["Buy"], df["Sell"]=signals()


#Plotting
plt.figure(figsize=(17,11))
plt.plot(df["SMA"], label=f"SMA {t}", linestyle="--")
plt.title(f"Bollinger Bands for {longname}")
plt.plot(df["Close"], label="Stock Price", alpha=0.4, color="black")
plt.plot(df["Upper"], label="Upper Band", color="darkblue")
plt.plot(df["Lower"], label="Lower Band", color="red")
plt.fill_between(df.index, df["Lower"], df["Upper"], where=df["Upper"]>df["Lower"], color="lightgreen")
plt.scatter(df.index, df["Buy"], label="Buy Price", marker="^", color="green")
plt.scatter(df.index, df["Sell"], label="Sell Price", marker="v", color="red")
plt.legend(loc="upper left")
plt.xlabel("Date")

plt.show()

Comparación con TA-Lib

In [ ]:
da=pd.DataFrame()
da["Upper"]=df["Upper"]
da["Middle"]=df["SMA"]
da["Lower"]=df["Lower"]

da["Upper (TA-Lib)"], da["Middle (TA-Lib)"], da["Lower (TA-Lib)"]=ta.BBANDS(df["Close"], t, 2, 2)

da.dropna(inplace=True)


#Plotting
plt.figure(figsize=(17,11))
plt.subplot(211)
plt.title(f"Bollinger Bands for {longname}")
plt.plot(da["Upper"], label="Upper Band", color="darkblue")
plt.plot(df["Close"], label="Stock Price", alpha=0.5)
plt.plot(df["SMA"], label=f"SMA {t}", linestyle="--")
plt.plot(da["Lower"], label="Lower Band", color="red")
plt.fill_between(da.index, da["Lower"], da["Upper"], where=da["Upper"]>da["Lower"], color="lightgreen")
plt.legend(loc="upper left")

plt.subplot(212)
plt.title("Bollinger Bands (TA-Lib)")
plt.plot(da["Upper (TA-Lib)"], label="Upper Band (TA-Lib)", color="darkblue")
plt.plot(df["Close"], label="Stock Price", alpha=0.5)
plt.plot(da["Middle (TA-Lib)"], label=f"SMA {t}", linestyle="--")
plt.plot(da["Lower (TA-Lib)"], label="Lower Band (TA-Lib)", color="red")
plt.xlabel("Date")
plt.fill_between(da.index, da["Lower (TA-Lib)"], da["Upper (TA-Lib)"], where=da["Upper (TA-Lib)"]>da["Lower (TA-Lib)"], color="lightgreen")
plt.legend(loc="upper left")

plt.show()
print(da.head(20))

##Coding 'CCI' Indicator

In [ ]:
df=yf.download(ticker, tf)


#Cálculo de 'CCI'
t=14
ty_p=(df["Close"]+df["High"]+df["Low"])/3

sma_typ=ty_p.rolling(t).mean()
stdev=ty_p.rolling(t).std()

df["CCI"]=(ty_p-sma_typ)/(0.015*stdev)


#Plotting
plt.figure(figsize=(17,11))
plt.subplot(211)
plt.plot(df['Close'], label="Stock Price")
plt.title(f"{longname} Stock Price")
plt.legend(loc="upper left")

plt.subplot(212)
plt.title("CCI chart")
plt.plot(df["CCI"])
plt.xlabel("Date")
plt.fill_between(df.index, df["CCI"], 100, where=df["CCI"]>100, color="lightcoral")
plt.fill_between(df.index, df["CCI"], -100, where=df["CCI"]<-100, color="lightgreen")
plt.axhline(100, linestyle='--', color="black",alpha=0.8)
plt.axhline(-100, linestyle='--', color="black",alpha=0.8)

plt.show()

Comparación con TA-Lib

In [ ]:
da=pd.DataFrame()
da["CCI"]=df["CCI"]

da["CCI (TA-Lib)"]=ta.CCI(df["High"], df["Low"], df["Close"], t)


#Plotting
plt.figure(figsize=(17,11))
plt.subplot(211)
plt.plot(df["CCI"], label="CCI")
plt.title("CCI Comparison")
plt.legend(loc="upper left")
plt.axhline(100, linestyle='--', color="black",alpha=0.8)
plt.axhline(-100, linestyle='--', color="black",alpha=0.8)


plt.subplot(212)
plt.title("CCI TA-Lib")
plt.plot(da["CCI (TA-Lib)"], label="CCI (TA-Lib)")
plt.xlabel("Date")
plt.legend(loc="upper left")
plt.axhline(100, linestyle='--', color="black",alpha=0.8)
plt.axhline(-100, linestyle='--', color="black",alpha=0.8)

plt.show()
print(da.tail(20))

##Coding 'Fibonacci Retracement'

In [ ]:
df=yf.download(ticker, "2020-6-1", "2022-3-25")


#Plotting
plt.figure(figsize=(17,11))
plt.title(f"Tesla Stock Price")
plt.plot(df["Close"], label="Stock Price")
plt.legend(loc="upper left")
plt.xlabel("Date")

plt.show()


#Establecemos la Swing
df1=yf.download(ticker, "2021-3-15", "2022-3-25")
maximum=df1["Close"].max()
minimum=df1["Close"].min()


#Algoritmo para establecer los límites de la Swing
def ptos():
    a=[]
    b=[]
    c=[]
    p=[]
    x=0
    for i in range(len(df1)):
        if df1["Close"][i]==maximum:
            a+=[df1["Close"][i]]
            b+=[np.nan]
            for j in range(i+1,len(df1)):
                p+=[df1["Close"][j]]
            x=min(p)
        elif df1["Close"][i]==minimum:
            b+=[df1["Close"][i]]
            a+=[np.nan]
        else:
            a+=[np.nan]
            b+=[np.nan]

    for i in range(len(df1)):
        if df1["Close"][i]==x:
            c+=[x]
        else:
            c+=[np.nan]
    return a, b, c

df1["Máximo"], df1["Mínimo"], df1["Pullback"]=ptos()


#Establecemos los distintos niveles
level_1=maximum-(0.236*(maximum-minimum))
level_2=maximum-(0.382*(maximum-minimum))
level_3=maximum-(0.5*(maximum-minimum))
level_4=maximum-(0.618*(maximum-minimum))
level_5=maximum-(0.784*(maximum-minimum))


#Plotting
plt.figure(figsize=(17,11))
plt.title(f"Tesla Stock Price")
plt.plot(df1["Close"], label="Stock Price", alpha=0.7)
plt.suptitle("Fibonacci Retracement", y=0.94, x=0.51)
plt.axhline(maximum, xmin=0.612, xmax=0.95, linestyle="--", alpha=0.5, color="black")
plt.axhline(level_1, xmin=0.612, xmax=0.95, linestyle="--", alpha=0.6, color="darkgreen")
plt.axhline(level_2, xmin=0.612, xmax=0.95, linestyle="--", alpha=0.6, color="green")
plt.axhline(level_3, xmin=0.612, xmax=0.95, linestyle="dotted", alpha=0.5, color="black")
plt.axhline(level_4, xmin=0.612, xmax=0.95, linestyle="--", alpha=0.6, color="green")
plt.axhline(level_5, xmin=0.612, xmax=0.95, linestyle="--", alpha=0.6, color="darkgreen")
plt.axhline(minimum, xmin=0.612, xmax=0.95, linestyle="--", alpha=0.5, color="black")
plt.scatter(df1.index, df1["Máximo"], label="Swing Max (2021-11-03)", color="red")
plt.scatter(df1.index, df1["Mínimo"], label="Swing Min (2020-3-23)", color="green")
plt.scatter(df1.index, df1["Pullback"], label="Swing Pullback", color="black")
plt.legend(loc="upper left")
plt.xlabel("Date")

plt.show()

##Coding 'Ichimoku Cloud' Indicator

In [ ]:
df=yf.download(ticker, tf)


#Cálculo 'Tenkan-sen'
t=9
max_9=df["High"].rolling(t).max()
min_9=df["Low"].rolling(t).min()
df["Tenkan-Sen"]=((max_9+min_9)/2)


#Cálculo 'Kijun-sen'
t=26
max_26=df["High"].rolling(t).max()
min_26=df["Low"].rolling(t).min()
df["Kijun-Sen"]=((max_26+min_26)/2)


#Cálculo 'Senkou Span A'
df["Senkou Span A"]=((df["Tenkan-Sen"]+df["Kijun-Sen"])/2).shift(26)


#Cálculo 'Senkou Span B'
t=52
max_52=df["High"].rolling(t).max()
min_52=df["Low"].rolling(t).min()
df["Senkou Span B"]=((max_52+min_52)/2).shift(26)


#Cálculo 'Chikou Span'
df["Chikou Span"]=df["Close"].shift(-26)


#Algoritmo de c/v
def signals():
    b=[]
    s=[]
    for i in range(len(df)):
        if i==0:
            b+=[np.nan]
            s+=[np.nan]
        elif df["Tenkan-Sen"][i-1]<df["Kijun-Sen"][i-1] and df["Tenkan-Sen"][i]>df["Kijun-Sen"][i]:
            b+=[df["Close"][i]]
            s+=[np.nan]
        elif df["Tenkan-Sen"][i-1]>df["Kijun-Sen"][i-1] and df["Tenkan-Sen"][i]<df["Kijun-Sen"][i]:
            b+=[np.nan]
            s+=[df["Close"][i]]
        else:
            b+=[np.nan]
            s+=[np.nan]
    return b, s

df["Buy"], df["Sell"]=signals()


#Plotting
plt.figure(figsize=(17,11))
plt.suptitle(f"Ichimoku Cloud for {longname}", y=0.94, x=0.51)
plt.plot(df["Close"], label="Stock Price", lw=1.3)
plt.plot(df["Tenkan-Sen"], lw=0.9, label="Tenkan-Sen", color="pink")
plt.plot(df["Kijun-Sen"], lw=0.9, label="Kijun-Sen", color="brown")
plt.plot(df["Senkou Span A"], lw=0.9, label="Senkou Span A", color="green")
plt.plot(df["Senkou Span B"], lw=0.9, label="Senkou Span B", color="red")
plt.plot(df["Chikou Span"], lw=0.9, label="Chikou Span", color="darkblue")
plt.legend(loc="upper left")
plt.title(f"{longname} Stock Price")
plt.fill_between(df.index, df["Senkou Span A"], df["Senkou Span B"], where=df["Senkou Span A"]>df["Senkou Span B"], color="lightgreen")
plt.fill_between(df.index, df["Senkou Span A"], df["Senkou Span B"], where=df["Senkou Span A"]<df["Senkou Span B"], color="lightcoral")
plt.xlabel("Date")

plt.show()

plt.figure(figsize=(17,11))
plt.suptitle("Ichimoku Cloud TK Cross Strategy")
plt.plot(df["Close"], label="Stock Price", alpha=0.8)
plt.plot(df["Tenkan-Sen"], lw=0.9, label="Tenkan-Sen", color="pink", alpha=0.2)
plt.plot(df["Kijun-Sen"], lw=0.9, label="Kijun-Sen", color="brown", alpha=0.2)
plt.plot(df["Senkou Span A"], lw=0.9, label="Senkou Span A", color="green", alpha=0.2)
plt.plot(df["Senkou Span B"], lw=0.9, label="Senkou Span B", color="red", alpha=0.2)
plt.plot(df["Chikou Span"], lw=0.9, label="Chikou Span", color="darkblue", alpha=0.2)
plt.scatter(df.index, df["Buy"], label="Buy Price", marker="^", color="green")
plt.scatter(df.index, df["Sell"], label="Sell Price", marker="v", color="red")
plt.legend(loc="upper left")
plt.title(f"{longname} Stock Price")
plt.fill_between(df.index, df["Senkou Span A"], df["Senkou Span B"], where=df["Senkou Span A"]>df["Senkou Span B"], color="lightgreen")
plt.fill_between(df.index, df["Senkou Span A"], df["Senkou Span B"], where=df["Senkou Span A"]<df["Senkou Span B"], color="lightcoral")
plt.xlabel("Date")

plt.show()

##Coding 'MACD' Indicator

In [ ]:
data=yf.download(ticker,"2020-6-1")["Close"]

df=pd.DataFrame()
df["Close"]=data


#Cálculo de 'EMA'
t_slow=26
t_fast=12
t_signal=9
df["EMA Slow"]=df["Close"].ewm(span=t_slow,adjust=False, min_periods=t_slow).mean()
df["EMA Fast"]=df["Close"].ewm(span=t_fast,adjust=False, min_periods=t_fast).mean()

#Cálculo de 'MACD'
df["MACD"]=df["EMA Fast"]-df["EMA Slow"]

#Cálculo de 'Signal'
df["Signal"]=df["MACD"].ewm(span=t_signal ,adjust=False, min_periods=t_signal).mean()

#Cálculo de 'Histograma'
df["Hist"]=df["MACD"]-df["Signal"]


#Algoritmo de c/v
def signals():
    b=[]
    s=[]
    for i in range(len(df)):
        if i!=0:
            if df["MACD"][i-1]<0 and df["MACD"][i]>0:
                b+=[df["Close"][i]]
                s+=[np.nan]
            elif df["MACD"][i-1]>0 and df["MACD"][i]<0:
                b+=[np.nan]
                s+=[df["Close"][i]]
            else:
                b+=[np.nan]
                s+=[np.nan]
        else:
            b+=[np.nan]
            s+=[np.nan]
    return b,s

df["Buy"],df["Sell"]=signals()


#Plotting
plt.figure(figsize=(17,11))
plt.subplot(211)
plt.plot(df["Close"], label="Stock Price", alpha=0.7)
plt.scatter(df.index, df["Buy"], label="Buy Price", marker="^", color="green")
plt.scatter(df.index, df["Sell"], label="Sell Price", marker="v", color="red")
plt.title(f"{longname} Stock Price")
plt.legend(loc="upper left")

plt.subplot(212)
plt.plot(df["Signal"], color="red", linestyle="--", label="Signal")
plt.plot(df["MACD"], color="darkblue", label="MACD")
plt.plot(df["Hist"], color="black", alpha=0.5)
plt.axhline(0, color="black", lw=0.5)
plt.title("MACD Chart")
plt.xlabel("Date")
plt.legend(loc="upper left")

plt.show()

Comparación con TA-Lib

In [ ]:
da=pd.DataFrame()

da["MACD"]=df["MACD"]
da["Signal"]=df["Signal"]
da["Hist"]=df["Hist"]

da["MACD (TA-Lib)"], da["Signal (TA-Lib)"], da["Hist (TA-Lib)"]=ta.MACD(df["Close"], t_fast, t_slow, t_signal)
da.dropna(inplace=True)


#Plotting
plt.figure(figsize=(17,11))
plt.subplot(211)
plt.plot(da["Signal"], color="red", linestyle="--", label="Signal")
plt.plot(da["MACD"], color="darkblue", label="MACD")
plt.plot(da["Hist"], color="black", alpha=0.5)
plt.axhline(0, color="black", lw=0.5)
plt.title("MACD Chart")
plt.legend(loc="upper left")

plt.subplot(212)
plt.plot(da["Signal (TA-Lib)"], color="red", linestyle="--", label="Signal (TA-Lib)")
plt.plot(da["MACD (TA-Lib)"], color="darkblue", label="MACD (TA-Lib)")
plt.plot(da["Hist (TA-Lib)"], color="black", alpha=0.5)
plt.axhline(0, color="black", lw=0.5)
plt.title("MACD (TA-Lib) Chart")
plt.legend(loc="upper left")
plt.xlabel("Date")

plt.show()
print(da.head(30))

##Coding 'RSI' Indicator

In [ ]:
data=yf.download(ticker, tf)

df=pd.DataFrame()
df["Close"]=data["Close"]
df["High"]=data["High"]
df["Low"]=data["Low"]


#Cálculo 'RSI'
t_rsi=14

df["Diff"]=df["Close"].diff(1)

def var():
    var_pos=[]
    var_neg=[]
    for i in range(len(df)):
        if df["Diff"][i]>0:
            var_pos+=[df["Diff"][i]]
            var_neg+=[0]
        elif df["Diff"][i]<0:
            var_neg+=[abs(df["Diff"][i])]
            var_pos+=[0]
        else:
            var_pos+=[0]
            var_neg+=[0]
    return var_pos,var_neg
pos,neg=var()

df["Var+"]=pos
df["Var-"]=neg

df["Prom+"]=df["Var+"].ewm(t_rsi,adjust=False, min_periods=t_rsi).mean()
df["Prom-"]=df["Var-"].ewm(t_rsi,adjust=False, min_periods=t_rsi).mean()

df["RS"]=df["Prom+"]/df["Prom-"]

df["RSI"]=100-(100/(1+df["RS"]))


#Plotting 
plt.figure(figsize=(17,11))
plt.subplot(211)
plt.plot(df['Close'], label="Stock Price")
plt.title(f"{longname} Stock Price")
plt.legend(loc="upper left")

plt.subplot(212)
plt.title("RSI chart")
plt.plot(df["RSI"])
plt.xlabel("Date")
plt.fill_between(df.index, df["RSI"], 70, where=df["RSI"]>70, color="lightcoral")
plt.fill_between(df.index, df["RSI"], 30, where=df["RSI"]<30, color="lightgreen")
plt.axhline(30, linestyle="--", color="black",alpha=0.8)
plt.axhline(50, linestyle="--", color="black", alpha=0.5)
plt.axhline(70, linestyle="--", color="black",alpha=0.8)

plt.show()

Comparación con TA-Lib

In [ ]:
da=pd.DataFrame()
da["RSI"]=df["RSI"]

da["RSI (TA-Lib)"]=ta.RSI(df["Close"], t_rsi)

da.dropna(inplace=True)


#Plotting
plt.figure(figsize=(17,11))
plt.subplot(211)
plt.title("RSI Comparison")
plt.plot(da["RSI"], label="RSI")
plt.axhline(30, linestyle="--", color="black",alpha=0.8)
plt.axhline(50, linestyle="--", color="black", alpha=0.5)
plt.axhline(70, linestyle="--", color="black",alpha=0.8)
plt.legend(loc="upper left")

plt.subplot(212)
plt.title("RSI (TA-Lib)")
plt.plot(da["RSI (TA-Lib)"], label="RSI (TA-Lib)", color="brown")
plt.axhline(30, linestyle="--", color="black",alpha=0.8)
plt.axhline(50, linestyle="--", color="black", alpha=0.5)
plt.axhline(70, linestyle="--", color="black",alpha=0.8)
plt.legend(loc="upper left")
plt.xlabel("Date")

plt.show()
print(da.head(25))

##Coding 'Stochastic Oscillator' Indicator

In [ ]:
df=yf.download(ticker, tf)


#Cálculo de 'Fast %K'
t_k=14
t_d=3
max_14=df["High"].rolling(t_k).max()
min_14=df["Low"].rolling(t_k).min()

k=((df["Close"]-min_14)/(max_14-min_14))*100


#Cálculo de 'Fast %D'
d=k.rolling(t_d).mean()


#Cálculo de 'Slow %K'
slow_k=k.rolling(t_d).mean()


#Cálculo de 'Slow %D'
slow_d=slow_k.rolling(t_d).mean()


#Plotting
def chart(x):
    plt.figure(figsize=(17,11))
    plt.subplot(211)
    plt.plot(df["Close"], label="Close Price")
    plt.title(f"{longname} Stock Price")
    plt.legend(loc="upper left")

    plt.subplot(212)
    plt.title(f"{x} Stochastic Oscillator")
    plt.axhline(20, linestyle="--", alpha=0.6, color="black")
    plt.axhline(50, linestyle="--", alpha=0.4, color="black")
    plt.axhline(80, linestyle="--", alpha=0.6, color="black")
    if x=="Fast":
        plt.plot(k, label="Fast %K")
        plt.plot(d, label="Fast %D")
        plt.fill_between(df.index, k, 20, where=k<20, color="lightgreen")
        plt.fill_between(df.index, k, 80, where=k>80, color="lightcoral")
    else:
        plt.plot(slow_k, label="Slow %K")
        plt.plot(slow_d, label="Slow %D")
        plt.fill_between(df.index, slow_k, 20, where=slow_k<20, color="lightgreen")
        plt.fill_between(df.index, slow_k, 80, where=slow_k>80, color="lightcoral")

    plt.xlabel("Date")
    plt.legend(loc="upper left")

    plt.show()

#Fast
chart("Fast")

#Slow
chart("Slow")

Comparación con TA-Lib

In [ ]:
da=pd.DataFrame()

da["Slow %K"]=slow_k
da["Slow %D"]=slow_d
da["Slow %K (TA-Lib)"], da["Slow %D (TA-Lib)"]=ta.STOCH(df["High"], df["Low"], df["Close"], t_k, t_d)
da["Fast %K"]=k
da["Fast %D"]=d
da["Fast %K (TA-Lib)"], da["Fast %D (TA-Lib)"]=ta.STOCHF(df["High"], df["Low"], df["Close"], t_k, t_d)
da.dropna(inplace=True)


#Plotting
def chart_ta(y):

    plt.figure(figsize=(17,11))
    plt.subplot(211)
    plt.title(f"{y} Stochastic Oscillator")
    plt.axhline(20, linestyle="--", alpha=0.6, color="black")
    plt.axhline(50, linestyle="--", alpha=0.4, color="black")
    plt.axhline(80, linestyle="--", alpha=0.6, color="black")

    if y=="Fast":

        plt.plot(k, label="Fast %K") 
        plt.plot(d, label="Fast %D")

    else:

        plt.plot(slow_k, label=f"{y} %K") 
        plt.plot(slow_d, label=f"{y} %D")
    
    plt.legend(loc="upper left")


    plt.subplot(212)
    plt.title(f"{y} Stochastic Oscillator (TA-Lib)")
    plt.axhline(20, linestyle="--", alpha=0.6, color="black")
    plt.axhline(50, linestyle="--", alpha=0.4, color="black")
    plt.axhline(80, linestyle="--", alpha=0.6, color="black")

    if y=="Fast":

        plt.plot(da["Fast %K (TA-Lib)"], label="Fast %K (TA-Lib)") 
        plt.plot(da["Fast %D (TA-Lib)"], label="Fast &D (TA-Lib)")

    else:

        plt.plot(da[f"{y} %K (TA-Lib)"], label="Slow %K (TA-Lib)") 
        plt.plot(da[f"{y} %D (TA-Lib)"], label="Slow %D (TA-Lib)")

    plt.legend(loc="upper left")
    plt.xlabel("Date")

    plt.show()

#Fast
chart_ta("Fast")
print()
print("*"*15)

#Slow
chart_ta("Slow")

print(da.head(10))

##Coding 'Williams Percentage Range' Indicator

In [ ]:
df=yf.download(ticker, tf)


#Cálculo de 'Williams % Range'
t=21
max_14=df["High"].rolling(t).max()
min_14=df["Low"].rolling(t).min()
wr=(-(max_14-df["Close"])/(max_14-min_14))*100
df["W%R"]=wr

df.dropna(inplace=True)


#Plotting
plt.figure(figsize=(17,11))
plt.subplot(211)
plt.plot(df["Close"], label="Stock Price")
plt.title(f"{longname} Stock Price")
plt.legend(loc="upper left")
plt.grid()

plt.subplot(212)
plt.title("Williams % Range Chart")
plt.plot(df["W%R"])
plt.axhline(-20, linestyle="--", alpha=0.8, color="black")
plt.axhline(-50, linestyle="--", alpha=0.5, color="black")
plt.axhline(-80, linestyle="--", alpha=0.8, color="black")
plt.fill_between(df.index, df["W%R"], -20, where=df["W%R"]>-20, color="coral")
plt.fill_between(df.index, df["W%R"], -80, where=df["W%R"]<-80, color="lightgreen")
plt.xlabel("Date")

plt.show()

Comparación con TA-Lib

In [ ]:
da=pd.DataFrame()
da["W%R"]=df["W%R"]

da["W%R (TA-Lib)"]=ta.WILLR(df["High"], df["Low"], df["Close"],21)
da.dropna(inplace=True)


plt.figure(figsize=(17,11))
plt.title("W%R Comparison")
plt.plot(da["W%R"], label="W%R")
plt.plot(da["W%R (TA-Lib)"], label="W%R (TA-Lib)")
plt.xlabel("Date")
plt.legend(loc="upper left")
plt.axhline(-20, linestyle="--", alpha=0.8, color="black")
plt.axhline(-50, linestyle="--", alpha=0.5, color="black")
plt.axhline(-80, linestyle="--", alpha=0.8, color="black")
plt.xlabel("Date")

plt.show()
print(da.head(20))